In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

In [ ]:
# GPU selection (memory)
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
data_txt = open('./data_split/data_dir_dict_0614.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [ ]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [ ]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    tr_data[i,:] = img2.flatten()
    tr_label[i] = tr_data_dir['train'][i][1]

In [ ]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [ ]:
def cnn_model(x, n_class, dropout, reuse, is_training, width=512, height=512):
    with tf.variable_scope('ConvNet', reuse=reuse):
        # RGB data, 3-channel
        input_layer = tf.reshape(x, shape=[-1, height, width, 3], name='input_layer')
        
        k_init = tf.contrib.layers.xavier_initializer_conv2d()
        
        conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=48,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv1')
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name='pool1')
        # height/2, width/2
        
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv2')
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name='pool2')
        # height/4, width/4
        
        conv3 = tf.layers.conv2d(
            inputs=pool2,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv3')
        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2, name='pool3')
        # height/8, width/8
        
        conv4 = tf.layers.conv2d(
            inputs=pool3,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv4')
        pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2, name='pool4')
        # height/16, width/16
        
        conv5 = tf.layers.conv2d(
            inputs=pool4,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv5')
        pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2, name='pool5')
        # height/32, width/32
        
        #pool5_flat = tf.reshape(pool5, [-1, int(height/32) * int(width/32) * 96])
        pool5_flat = tf.contrib.layers.flatten(pool5)
        
        dense1 = tf.layers.dense(pool5_flat, 1024, activation=tf.nn.relu, name='dense1')
        
        dropout = tf.layers.dropout(dense1, rate=dropout, training=is_training, name='drop')
        
        # Output
        out = tf.layers.dense(dropout, n_class, name='output')
        out = tf.nn.softmax(out) if not is_training else out
        
        return out

In [ ]:
dropout = 0.5
learning_rate = 0.0001
batch_size = 16
disp_step = 20
save_step = 500
n_classes = 2
epochs = 30

tr_steps = int(tr_data.shape[0]/batch_size*epochs)
inner_steps = int((tr_data.shape[0])/batch_size)

In [ ]:
with tf.device('/cpu:0'):
    reuse_vars = False
    
    # tf graph input
    X = tf.placeholder(tf.float32, [None, total_pix], name='inputs')
    Y = tf.placeholder(tf.int32, [None, 1], name='labels')
    
    with tf.device('/gpu:1'):
        # cnn_model(x, n_class, dropout, re_use, is_training, width, height)
        logits_train = cnn_model(X, n_classes, dropout, reuse=reuse_vars,
                                 is_training=True)
        
        logits_test = cnn_model(X, n_classes, dropout, reuse=True,
                                is_training=False)
        
        loss = tf.losses.sparse_softmax_cross_entropy(logits=logits_train, labels=Y)
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        grads = optimizer.compute_gradients(loss)
        
        pred = tf.argmax(input=logits_test, axis=1)
        correct_pred = tf.equal(pred, tf.cast(Y, tf.int64))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        #accuracy = tf.metrics.accuracy(labels=Y, predictions=pred)
        
        
        prob = tf.nn.softmax(logits_test, name='softmax_tensor')
        
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        
        reuse_vars = True
        
    
    
    
    
    saver = tf.train.Saver()
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()

In [ ]:
    with tf.Session() as sess:

        sess.run(init_g)
        sess.run(init_l)

        file_n = '/models/ICROS/GPU1(1)'
        t_count = 1
        saver.save(sess, file_n)
        for j in range(epochs):

            for i in range(inner_steps):

                tr_batch = {}

                if i < inner_steps-1:
                    tr_batch['data'] = tr_data[i*batch_size:(i+1)*batch_size][:]
                    tr_batch['label'] = tr_label[i*batch_size:(i+1)*batch_size][:]

                else:
                    tr_batch['data'] = tr_data[i*batch_size:][:]
                    tr_batch['label'] = tr_label[i*batch_size:][:]

                sess.run(train_op, feed_dict={X: tr_batch['data'],
                                              Y: tr_batch['label']})

                if t_count % disp_step == 1:
                    
                    loss_tr = sess.run(loss, feed_dict = {X: tr_batch['data'],
                                                          Y: tr_batch['label']})

                    acc_tr = sess.run(accuracy, feed_dict = {X: tr_batch['data'],
                                                             Y: tr_batch['label']})
                    
                    prob_tr = sess.run(prob, feed_dict = {X: tr_batch['data'],
                                                         Y: tr_batch['label']})

                    #print('step %d, training accuracy %f, loss %f' % (t_count, accuracy, loss))
                    print('Training step: ', t_count)
                    print('Accuracy', acc_tr)
                    print('Probability')
                    print(prob_tr)
                    print('Loss: ', loss_tr)
                    #print(acc_tr, loss_tr)
                    
                if t_count % save_step == 0:
                    saver.save(sess, file_n, t_count)

                t_count = t_count+1
        
        saver.save(sess, file_n, t_count)
        print("Training is finished!!!")
        
        
        
        ev_accuracy = np.zeros((len(ev_label), 1))
        for k in range(len(ev_label)):
            ev_batch = {}
            ev_batch['data'] = ev_data[k:(k+1)][:]
            ev_batch['label'] = ev_label[k:(k+1)]
            ev_accuracy[k] = sess.run(accuracy, feed_dict={X: ev_batch['data'],
                                                           Y: ev_batch['label']})

        print(np.mean(ev_accuracy))